# GameNight Bar Chart Race

## References
 - https://medium.com/dunder-data/create-a-bar-chart-race-animation-in-python-with-matplotlib-477ed1590096

- https://stackoverflow.com/questions/28931224/adding-value-labels-on-a-matplotlib-bar-chart

- https://github.com/dexplo/bar_chart_race/blob/master/bar_chart_race/_make_chart.py


- **Note**: May need to `brew install ffmpeg`
  - May need to add following line directly below `import matplotlib.pyplot as plt`
  `plt.rcParams['animation.ffmpeg_path'] = '/usr/local/bin/ffmpeg'`

In [1]:
# Standard libraries
import warnings
from datetime import datetime
from pathlib import Path

# Third-party libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Define colors for plots
colors = plt.cm.Dark2(range(6))

# Ignore warnings
warnings.filterwarnings("ignore")

In [2]:
data_path = Path('data/')

trivia_data = pd.read_csv(data_path/'weekly_stats.csv', index_col='Date', parse_dates=['Date'])

# Cummulative Stats
trivia_cumsum = trivia_data.cumsum()
trivia_cumsum['Round'] = trivia_data['Round']

In [3]:
def prepare_data(df, steps=50):
    
    # Smooth transistions
    df                   = df.reset_index()
    df.index             = df.index * steps
    last_idx             = df.index[-1] + 1
    df_expanded          = df.reindex(range(last_idx))
    
    df_expanded['Date']  = df_expanded['Date'].fillna(method='ffill')
    df_expanded['Round'] = df_expanded['Round'].fillna(method='ffill')
    
    df_expanded          = df_expanded.set_index('Date')
    df_rank_expanded     = df_expanded.rank(axis=1, method='first')
    
    df_expanded          = df_expanded.interpolate()
    df_rank_expanded     = df_rank_expanded.interpolate()
    
    return df_expanded, df_rank_expanded

In [4]:
df_expanded, df_rank_expanded = prepare_data(trivia_cumsum, steps=10)

In [5]:
def nice_axes(ax):
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    
def init():
    ax.clear()
    nice_axes(ax)
    ax.set_ylim(.2, 6.8)
    

def update(i):
    for bar in ax.containers:
        bar.remove()
    for text in ax.texts[:]:
        text.remove()

    
    # Labels should not have 'Round'
    labels = df_rank_expanded.columns
    
    y     = df_rank_expanded.iloc[i]
    width = df_expanded.iloc[i].drop('Round').astype('float64')
    
    ax.barh(y=y, width=width, color=colors, tick_label=labels, height=0.6)
    
    # Add some space to far right plot based on max x_val
    max_x_val = df_expanded.iloc[i].drop('Round').astype('float64').max() 
    ax.set_xlim(0,max_x_val* 1.10)

    ax.set_title('Game Night Cumulative Stats')
    ax.tick_params(axis='y', labelsize=11)  # label sizes (team name font)

    # Date, GamType, and Round box 
    date_str = df_expanded.index[i].strftime('%Y-%m-%d')
    label    = f"{date_str} {df_expanded.iloc[i]['Round']}"
    
    props = dict(boxstyle='round', facecolor='0.8', alpha=0.5)
    ax.text(0.75, 0.08, label, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props, clip_on=True)

    # Data labels at right of bar
    delta        = 0.01
    ha           = 'left'
    va           = 'center'
    n_bars       = len(labels)
    bar_location = df_rank_expanded.iloc[i].values
    top_filt     = (bar_location > 0) & (bar_location < n_bars + 1)
    bar_location = bar_location[top_filt]
    bar_length   = df_expanded.iloc[i].drop('Round').astype('float64').values[top_filt]

    zipped = zip(bar_length, bar_location)

    for x1,y1 in zipped:
        
        label = f'{x1:,.1f}'  # format number with comma and one decimal place

        xtext, ytext = ax.transLimits.transform((x1, y1))
        xtext += delta

        xtext, ytext = ax.transLimits.inverted().transform((xtext, ytext))
        ax.text(xtext, ytext, label, ha=ha, va=va, clip_on=True)

In [6]:
fig = plt.Figure(figsize=(15, 7), dpi=150) # dpi is dots per inch

ax = fig.add_subplot()

anim = FuncAnimation(
    fig       = fig,
    func      = update,
    init_func = init,
    frames    = len(df_expanded), 
    interval  = 100,
    repeat    = False
)


html = anim.to_html5_video()
HTML(html)

In [7]:
generated_date = datetime.now().strftime('%Y-%m-%d')

anim.save(data_path/f'gamenight_barchart_race_{generated_date}.mp4')